In [ ]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from datetime import date


"""data has been saved as an xml file"""
import time
timestr = time.strftime("%Y%m%d-%H%M%S")


#import file name and string with date and time and save as variable
vic_fire_data = 'data/vic_data/vic_fire_data_'+timestr+'.xml'

#add the data from the website 
URL = "https://data.emergency.vic.gov.au/Show?pageId=getIncidentRSS"

#The binary mode (wb) is required to write the actual content of the XML page to your external file in the original format.
#will get the current up to date details from website
response = requests.get(URL)
with open(vic_fire_data , 'wb') as file:
    file.write(response.content)

#parse through the data
mytree = ET.parse(vic_fire_data) 
myroot = mytree.getroot()

all_desc_list =[]
for all_description in myroot.findall("./channel/item/description"):
    all_desc_list.append(all_description.text)

#removes tags that aren't needed. Is there a better way to append more items. Can probably do a if/else statement
clean_1 = []
for string in all_desc_list:
    clean_1.append(string.replace('</strong>',' '))
    
clean_2 = []
for string in clean_1:
    clean_2.append(string.replace('<strong>',' '))

clean_3 = []
for string in clean_2:
    clean_3.append(string.replace('<br>',' '))

clean_data = []
for entry in clean_3:
    clean_data.append(entry.split())

#function to get lat co_ords
vic_latitude_list = []
def vic_lat_co_ordinates(data):
    for entry in clean_data:
        vic_latitude_list.append(entry[-3])
    return vic_latitude_list 
vic_lat_co_ordinates(clean_data)
       
#function to get long co_ords
vic_longitude_list = []
def vic_long_co_ordinates(data):
    for entry in clean_data:
        vic_longitude_list.append(entry[-1])
    return vic_longitude_list    
vic_long_co_ordinates(clean_data) 

#add todays dated
today = date.today()
vic_date = today.strftime("%d/%m/%Y")


vic_df = pd.DataFrame({
    'Latitude': vic_latitude_list,
    'Longitude': vic_longitude_list,
    'State': 'VIC',
    'Report Date': vic_date
})                         

#append to the saved fire_data csv file

vic_df.to_csv("data/fire_data.csv", mode = 'a', index = False , header = False)

#remove duplicate values
fire_data_df = pd.read_csv('data/fire_data.csv')

#drop the duplicates out of the data frame
fire_data_df_no_duplicates = fire_data_df.drop_duplicates()

#create a new csv file with the dropped duplicate values
fire_data_df_no_duplicates.to_csv('data/fire_data_df_no_duplicates.csv',index = False)
display(fire_data_df_no_duplicates)